In [1]:
from __future__ import print_function, division

import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob

from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
import time

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []
    i = 0
    for file in glob.glob("bts_data/*.mid"):
        midi = converter.parse(file)
        i = i + 1
        if i > 5: break
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 250

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        print('[train]: Starting...')
        time_start = time.time()
        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Insert saving; The generator image is saved every 500 steps
        save_interval = 500

        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)



            if (epoch + 1) % save_interval == 0:

                # Save the generator model in a periodic basis
                model_name = './bts_models/bts_' + str(epoch)
                self.generator.save(model_name + ".h5")

                # Generate a midi file in a periodic basis
                self.generate(notes, epoch)


            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [time: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss, time.time() - time_start))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes, epoch)
        self.plot_loss()


        time_end = time.time()
        print('[train]: Ended. Time elapsed: {}'.format(time_end - time_start))
        
    def generate(self, input_notes, epoch):
        # Get pitch names and store in a dictionary
        print('[generate midi]: Starting...')
        time_start = time.time()
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        #print(pitchnames)
        #print(len(pitchnames))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        #print(len(int_to_note))
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        #print(predictions[0])
        pred_notes = []


        old_max = 1
        old_min = -1
        new_max = 41
        new_min = 0
        old_range = old_max - old_min
        new_range = new_max - new_min
        for i, x in enumerate(predictions[0]):
            entry = (((x - old_min)* new_range)/old_range) + new_min
            # if x < 0:
            #     entry = (x+1)*21
            # else:
            #     entry = (x*21) + 21
            pred_notes.append(entry)
        #pred_notes = [int((x+1)*89) for x in predictions[0]]
        #pred_notes = [x*242+242 for x in predictions[0]]
        #print(len(int_to_note))
        #print(pred_notes)
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        

        file_name = './bts_output/bts_' + str(epoch)
        create_midi(pred_notes, file_name)
        time_end = time.time()
        print('[generate midi]: Ended. Time elapsed: {}'.format(time_end - time_start))

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('./bts_output/GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

if __name__ == '__main__':
  gan = GAN(rows=250)    
  gan.train(epochs=3000, batch_size=32, sample_interval=1)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 250, 512)          1054720   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dens

/home/dlstudent02/anaconda3/envs/coe197/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.698065, acc.: 21.88%] [G loss: 0.685202] [time: 11.047449]
1 [D loss: 0.695017, acc.: 50.00%] [G loss: 0.683770] [time: 11.282772]
2 [D loss: 0.692292, acc.: 50.00%] [G loss: 0.680532] [time: 11.506519]
3 [D loss: 0.695525, acc.: 50.00%] [G loss: 0.677414] [time: 11.729527]
4 [D loss: 0.693793, acc.: 50.00%] [G loss: 0.674674] [time: 11.952857]
5 [D loss: 0.695660, acc.: 50.00%] [G loss: 0.668426] [time: 12.178992]
6 [D loss: 0.697793, acc.: 50.00%] [G loss: 0.669197] [time: 12.404836]
7 [D loss: 0.692531, acc.: 50.00%] [G loss: 0.663462] [time: 12.630647]
8 [D loss: 0.702338, acc.: 50.00%] [G loss: 0.667122] [time: 12.852473]
9 [D loss: 0.701656, acc.: 50.00%] [G loss: 0.674510] [time: 13.078819]
10 [D loss: 0.689359, acc.: 54.69%] [G loss: 0.679700] [time: 13.302456]
11 [D loss: 0.694862, acc.: 50.00%] [G loss: 0.677151] [time: 13.530413]
12 [D loss: 0.705805, acc.: 48.44%] [G loss: 0.680907] [time: 13.752506]
13 [D loss: 0.696022, acc.: 40.62%] [G loss: 0.685487] [time:

113 [D loss: 0.693555, acc.: 40.62%] [G loss: 0.692630] [time: 36.544140]
114 [D loss: 0.693593, acc.: 40.62%] [G loss: 0.693151] [time: 36.772038]
115 [D loss: 0.693242, acc.: 42.19%] [G loss: 0.693585] [time: 36.994388]
116 [D loss: 0.693147, acc.: 46.88%] [G loss: 0.693024] [time: 37.222248]
117 [D loss: 0.692506, acc.: 48.44%] [G loss: 0.694827] [time: 37.447315]
118 [D loss: 0.693553, acc.: 43.75%] [G loss: 0.697799] [time: 37.673730]
119 [D loss: 0.691594, acc.: 51.56%] [G loss: 0.697625] [time: 37.898012]
120 [D loss: 0.695075, acc.: 51.56%] [G loss: 0.694121] [time: 38.122013]
121 [D loss: 0.697440, acc.: 35.94%] [G loss: 0.692353] [time: 38.345626]
122 [D loss: 0.693531, acc.: 37.50%] [G loss: 0.692578] [time: 38.567326]
123 [D loss: 0.692616, acc.: 56.25%] [G loss: 0.689960] [time: 38.791939]
124 [D loss: 0.694078, acc.: 46.88%] [G loss: 0.689129] [time: 39.014766]
125 [D loss: 0.694149, acc.: 43.75%] [G loss: 0.691307] [time: 39.241153]
126 [D loss: 0.694938, acc.: 45.31%] [

...

2502 [D loss: 0.694095, acc.: 40.62%] [G loss: 0.692921] [time: 581.406039]
2503 [D loss: 0.693513, acc.: 42.19%] [G loss: 0.692797] [time: 581.630275]
2504 [D loss: 0.694563, acc.: 35.94%] [G loss: 0.692201] [time: 581.858718]
2505 [D loss: 0.696071, acc.: 39.06%] [G loss: 0.691196] [time: 582.090346]
2506 [D loss: 0.693696, acc.: 43.75%] [G loss: 0.692587] [time: 582.318941]
2507 [D loss: 0.694968, acc.: 50.00%] [G loss: 0.691611] [time: 582.546084]
2508 [D loss: 0.695055, acc.: 35.94%] [G loss: 0.689106] [time: 582.772939]
2509 [D loss: 0.695752, acc.: 39.06%] [G loss: 0.690090] [time: 583.002375]
2510 [D loss: 0.692100, acc.: 46.88%] [G loss: 0.694079] [time: 583.234620]
2511 [D loss: 0.694568, acc.: 43.75%] [G loss: 0.702441] [time: 583.463018]
2512 [D loss: 0.699072, acc.: 39.06%] [G loss: 0.699617] [time: 583.689899]
2513 [D loss: 0.695624, acc.: 45.31%] [G loss: 0.692353] [time: 583.915771]
2514 [D loss: 0.695619, acc.: 46.88%] [G loss: 0.691006] [time: 584.144553]
2515 [D loss

2610 [D loss: 0.694913, acc.: 35.94%] [G loss: 0.693068] [time: 606.155535]
2611 [D loss: 0.694401, acc.: 40.62%] [G loss: 0.691801] [time: 606.385489]
2612 [D loss: 0.695767, acc.: 37.50%] [G loss: 0.691680] [time: 606.615158]
2613 [D loss: 0.694507, acc.: 39.06%] [G loss: 0.689820] [time: 606.843066]
2614 [D loss: 0.695152, acc.: 42.19%] [G loss: 0.692451] [time: 607.073287]
2615 [D loss: 0.692869, acc.: 39.06%] [G loss: 0.690649] [time: 607.299083]
2616 [D loss: 0.694416, acc.: 37.50%] [G loss: 0.691452] [time: 607.529316]
2617 [D loss: 0.694710, acc.: 40.62%] [G loss: 0.689951] [time: 607.759280]
2618 [D loss: 0.696229, acc.: 37.50%] [G loss: 0.692977] [time: 607.988877]
2619 [D loss: 0.695494, acc.: 39.06%] [G loss: 0.692099] [time: 608.218337]
2620 [D loss: 0.695434, acc.: 35.94%] [G loss: 0.693629] [time: 608.446581]
2621 [D loss: 0.696459, acc.: 35.94%] [G loss: 0.694076] [time: 608.676288]
2622 [D loss: 0.694305, acc.: 31.25%] [G loss: 0.694241] [time: 608.904093]
2623 [D loss

2718 [D loss: 0.693888, acc.: 48.44%] [G loss: 0.688003] [time: 630.839158]
2719 [D loss: 0.697970, acc.: 35.94%] [G loss: 0.691092] [time: 631.066693]
2720 [D loss: 0.693236, acc.: 34.38%] [G loss: 0.691070] [time: 631.294144]
2721 [D loss: 0.696014, acc.: 42.19%] [G loss: 0.691412] [time: 631.524383]
2722 [D loss: 0.693347, acc.: 39.06%] [G loss: 0.691643] [time: 631.753453]
2723 [D loss: 0.696829, acc.: 39.06%] [G loss: 0.690876] [time: 631.981517]
2724 [D loss: 0.695688, acc.: 40.62%] [G loss: 0.692617] [time: 632.209613]
2725 [D loss: 0.697741, acc.: 37.50%] [G loss: 0.694051] [time: 632.435991]
2726 [D loss: 0.698536, acc.: 23.44%] [G loss: 0.692521] [time: 632.663989]
2727 [D loss: 0.696746, acc.: 23.44%] [G loss: 0.692413] [time: 632.888970]
2728 [D loss: 0.694493, acc.: 42.19%] [G loss: 0.697849] [time: 633.119508]
2729 [D loss: 0.694973, acc.: 43.75%] [G loss: 0.699678] [time: 633.349971]
2730 [D loss: 0.694487, acc.: 48.44%] [G loss: 0.695540] [time: 633.576519]
2731 [D loss

2826 [D loss: 0.699104, acc.: 34.38%] [G loss: 0.698468] [time: 655.662466]
2827 [D loss: 0.699375, acc.: 40.62%] [G loss: 0.697376] [time: 655.891889]
2828 [D loss: 0.696237, acc.: 40.62%] [G loss: 0.693787] [time: 656.123239]
2829 [D loss: 0.695852, acc.: 39.06%] [G loss: 0.695056] [time: 656.351166]
2830 [D loss: 0.695141, acc.: 32.81%] [G loss: 0.693680] [time: 656.583445]
2831 [D loss: 0.695319, acc.: 32.81%] [G loss: 0.694225] [time: 656.814374]
2832 [D loss: 0.695098, acc.: 35.94%] [G loss: 0.693783] [time: 657.045516]
2833 [D loss: 0.695430, acc.: 40.62%] [G loss: 0.694299] [time: 657.274737]
2834 [D loss: 0.694779, acc.: 35.94%] [G loss: 0.694150] [time: 657.502291]
2835 [D loss: 0.693448, acc.: 42.19%] [G loss: 0.694327] [time: 657.732424]
2836 [D loss: 0.694852, acc.: 45.31%] [G loss: 0.694834] [time: 657.960623]
2837 [D loss: 0.694653, acc.: 40.62%] [G loss: 0.695356] [time: 658.188929]
2838 [D loss: 0.695714, acc.: 40.62%] [G loss: 0.693568] [time: 658.418255]
2839 [D loss

2934 [D loss: 0.697568, acc.: 51.56%] [G loss: 0.696450] [time: 680.365561]
2935 [D loss: 0.693863, acc.: 48.44%] [G loss: 0.693226] [time: 680.592235]
2936 [D loss: 0.694845, acc.: 39.06%] [G loss: 0.695953] [time: 680.818680]
2937 [D loss: 0.693676, acc.: 46.88%] [G loss: 0.697865] [time: 681.044189]
2938 [D loss: 0.692881, acc.: 53.12%] [G loss: 0.695750] [time: 681.272174]
2939 [D loss: 0.693274, acc.: 51.56%] [G loss: 0.697732] [time: 681.505157]
2940 [D loss: 0.694181, acc.: 42.19%] [G loss: 0.695769] [time: 681.733034]
2941 [D loss: 0.693047, acc.: 48.44%] [G loss: 0.697001] [time: 681.961692]
2942 [D loss: 0.693999, acc.: 51.56%] [G loss: 0.698840] [time: 682.188546]
2943 [D loss: 0.695086, acc.: 40.62%] [G loss: 0.698210] [time: 682.417461]
2944 [D loss: 0.694342, acc.: 50.00%] [G loss: 0.696686] [time: 682.645224]
2945 [D loss: 0.693270, acc.: 53.12%] [G loss: 0.697551] [time: 682.872339]
2946 [D loss: 0.695045, acc.: 42.19%] [G loss: 0.694316] [time: 683.097755]
2947 [D loss